## Web Scraping 

In [1]:
import numpy as np
import pandas as pd
import datetime 

from tqdm.notebook import tqdm

In [2]:
from web_scrapper import Parser

# TODO Manage SQL database
database = pd.read_pickle('df.pkl')

parser = Parser(data=database)
parser.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80656 entries, 0 to 80655
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   time       80656 non-null  datetime64[ns]
 1   title      80656 non-null  object        
 2   link       80656 non-null  object        
 3   source     80656 non-null  object        
 4   processed  80656 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 3.1+ MB


In [3]:
parser.parse(mode='update')
parser.df.info()

  0%|          | 0/3 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81675 entries, 0 to 81674
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   time       81675 non-null  datetime64[ns]
 1   title      81675 non-null  object        
 2   link       81675 non-null  object        
 3   source     81675 non-null  object        
 4   processed  80656 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 3.1+ MB


## Text Preprocessing

In [5]:
from preprocess import preprocess, fill_na

parser.df['processed'] = parser.df.apply(fill_na, axis=1)
parser.df.to_pickle('df.pkl')

parser.df.tail(10)

,time,title,link,source,processed
81665,2022-12-27 13:59:00,В Литве призвали поставлять еще больше оружия ...,https://lenta.ru/news/2022/12/27/nuzhda/,lenta,литва призывать поставлять оружие украина
81666,2022-12-27 14:00:00,Назван самый модный головной убор на зиму,https://lenta.ru/news/2022/12/27/balaclava/,lenta,называть самый модный головной убор зима
81667,2022-12-27 14:03:00,В МВД России сообщили о планах облегчить услов...,https://interfax.ru/russia/878822,interfax,мвд россия сообщать план облегчать условие пре...
81668,2022-12-27 14:03:00,Женатым мужчинам подсказали способ пресечь поп...,https://lenta.ru/news/2022/12/27/flirt/,lenta,женатый мужчина подсказывать способ пресекать ...
81669,2022-12-27 14:04:00,Минсельхоз не видит необходимости во введении ...,https://interfax.ru/business/878823,interfax,минсельхоз видеть необходимость введение потол...
81670,2022-12-27 14:05:00,В Госдуме высказались о готовности Украины к п...,https://lenta.ru/news/2022/12/27/peregg/,lenta,госдума высказываться готовность украина перег...
81671,2022-12-27 14:07:00,Шведской хоккеистке ответили на жалобу о прину...,https://lenta.ru/news/2022/12/27/otvet/,lenta,шведский хоккеистка отвечать жалоба принуждени...
81672,2022-12-27 14:08:00,В Китае прокомментировали открытие страны для ...,https://lenta.ru/news/2022/12/27/china_tourists/,lenta,китай прокомментировать открытие страна россиянин
81673,2022-12-27 14:08:00,В России оценили роль войсковой ПВО в ходе спе...,https://lenta.ru/news/2022/12/27/pvo/,lenta,россия оценивать роль войсковой пво ход спецоп...
81674,2022-12-27 14:10:00,Эксперты заявили о консолидации общества в отв...,https://lenta.ru/news/2022/12/27/experty/,lenta,эксперт заявлять консолидация общество ответ п...


## Extractor

In [6]:
data = parser.df.copy()

In [109]:
from sklearn.feature_extraction.text import CountVectorizer

class CountTrendExtractor():
    def __init__(self, df):
        
        self.data = df
        self.delta_dict = {
            '3hours': [10800, 32400],  # 3 hours, 9 hours
            '6hours': [21600, 64800],  # 6 hours, 18 hours
            'day': [86400, 518400],  # day, week
            'week': [604800, 1814400]  # week, month
        }
    def get_indexes(self, delta):
        """
        Get the indexes of two data slices.
        Parameters
        ----------
        delta: array of shape (1, 2)
            An array containing two deltas: the first is a time interval 
            (in seconds), which we will analyze as new data, the second is 
            the time interval preceding the first (in seconds)
        Returns
        -------
        new_indexes, old_indexes : tuple
            Tuple of the two Int64Index
        """
        start_date = datetime.datetime.now() - datetime.timedelta(seconds=delta[0])

        new_indexes = self.data.loc[self.data['time']
                                    >= pd.to_datetime(start_date)].index
        old_indexes = self.data.loc[(self.data['time'] <= pd.to_datetime(start_date)) & (
            self.data['time'] >= pd.to_datetime(start_date - datetime.timedelta(seconds=delta[1])))].index

        return new_indexes, old_indexes
    
    def get_top_mentioned(self, time_interval='week'):
        """
        Get the top relatively frequently mentioned words
        Parameters
        ----------
        time_interval: str
            Key from self.delta_dict: 'week', 'day', '6hours', '3hours'
        Returns
        -------
        res : dict of length vocab.shape
            Sorted by value dict of most frequent words
        """
        vectorizer = CountVectorizer(ngram_range=(1, 1))
        
        X = vectorizer.fit_transform(self.data['processed']).toarray()
        self.vocab = vectorizer.get_feature_names_out()
        
        new_indexes, old_indexes = self.get_indexes(self.delta_dict[time_interval])
        
        new_freq = np.asarray(X[new_indexes].sum(axis=0)).ravel() / new_indexes.shape[0]
        old_freq = np.asarray(X[old_indexes].sum(axis=0)).ravel() / old_indexes.shape[0]
        old_freq[old_freq == 0] = 1 / old_indexes.shape[0]
        
        razn = np.array(new_freq / old_freq)
        top = dict(zip(self.vocab, razn))
        res = sorted(top.items(), key=lambda item: item[1], reverse=True)
        
        return res

In [110]:
model = CountTrendExtractor(df = data)

model.get_top_mentioned()

[('кемерово', 121.86240913811008),
 ('престарелый', 118.65550363447561),
 ('приют', 54.517393561786086),
 ('рогозин', 48.10358255451713),
 ('кемеровский', 38.482866043613704),
 ('энгельс', 32.06905503634476),
 ('военкомат', 28.86214953271028),
 ('нрд', 28.86214953271028),
 ('бес', 22.44833852544133),
 ('танец', 22.44833852544133),
 ('футболистка', 19.241433021806852),
 ('макеевка', 17.637980269989615),
 ('shell', 16.03452751817238),
 ('возлюбленный', 16.03452751817238),
 ('задымление', 16.03452751817238),
 ('лавина', 16.03452751817238),
 ('призывной', 16.03452751817238),
 ('прогреметь', 16.03452751817238),
 ('рождество', 16.03452751817238),
 ('сахар', 16.03452751817238),
 ('тула', 16.03452751817238),
 ('цзиньпин', 16.03452751817238),
 ('экстрадиция', 16.03452751817238),
 ('матвиенко', 14.43107476635514),
 ('активизация', 12.827622014537903),
 ('бахмут', 12.827622014537903),
 ('буря', 12.827622014537903),
 ('бюджетный', 12.827622014537903),
 ('верфь', 12.827622014537903),
 ('вселяться',

In [46]:
new_freq = np.asarray(X[new_indexes].sum(axis=0)).ravel() / new_indexes.shape[0]
old_freq = np.asarray(X[old_indexes].sum(axis=0)).ravel() / old_indexes.shape[0]
old_freq[old_freq == 0] = 1 / old_indexes.shape[0]

vocab = vectorizer.get_feature_names_out()
razn = np.array(new_freq / old_freq)

In [47]:
top = dict(zip(vocab, razn))
sorted(top.items(), key=lambda item: item[1], reverse=True)

[('кемерово', 123.24273221460648),
 ('престарелый', 123.24273221460648),
 ('дом престарелый', 107.83739068778065),
 ('зеленский сша', 77.02670763412905),
 ('визит зеленский', 65.4727014890097),
 ('престарелый кемерово', 65.4727014890097),
 ('приют', 53.91869534389033),
 ('пожар дом', 46.21602458047742),
 ('гостиница донецк', 42.364689198770975),
 ('месторождение иркутский', 42.364689198770975),
 ('белгород', 38.51335381706453),
 ('кемеровский', 38.51335381706453),
 ('обстрел гостиница', 38.51335381706453),
 ('газопровод чувашия', 34.66201843535807),
 ('поездка зеленский', 34.66201843535807),
 ('зеленский вашингтон', 30.81068305365162),
 ('рогозин', 30.81068305365162),
 ('на', 28.885015362798395),
 ('взрыв газопровод', 26.959347671945164),
 ('газовый месторождение', 26.959347671945164),
 ('египет турист', 26.959347671945164),
 ('закон наказание', 26.959347671945164),
 ('солист группа', 26.959347671945164),
 ('больница донецк', 23.10801229023871),
 ('встречать', 23.10801229023871),
 ('вт

In [50]:
mask = df['processed'].str.contains(r'гадот', na=True)
df[mask]

,time,title,link,source,processed
76956,2022-12-19 13:22:00,Из «Флэша» вырежут камео Супермена и Чудо-женщ...,https://lenta.ru/news/2022/12/19/flashscenes/,lenta,флеш вырезать камео супермен чудо женщина кави...
77535,2022-12-20 11:25:00,Джеймс Ганн прокомментировал слухи об увольнен...,https://lenta.ru/news/2022/12/20/gann_gal/,lenta,джеймс ганн прокомментировать слух увольнение ...
79827,2022-12-23 12:41:00,Стало известно о возвращении Галь Гадот к роли...,https://lenta.ru/news/2022/12/23/gal_fast/,lenta,становиться известно возвращение галь гадот ро...


In [51]:
top

{'000': 0.0,
 '02': 1.9256676908532262,
 '02 годовой': 3.8513353817064524,
 '02 млрд': 0.0,
 '02 старт': 0.0,
 '04': 0.0,
 '04 баррель': 0.0,
 '05': 0.0,
 '05 бо': 0.0,
 '06': 0.0,
 '06 общий': 0.0,
 '088': 0.0,
 '088 человек': 0.0,
 '09': 0.0,
 '09 баррель': 0.0,
 '096': 0.0,
 '096 человек': 0.0,
 '10': 0.8502948245325933,
 '10 10': 0.0,
 '10 12': 0.0,
 '10 18': 3.8513353817064524,
 '10 20': 0.0,
 '10 23': 0.0,
 '10 26': 0.0,
 '10 75': 0.0,
 '10 iv': 0.0,
 '10 агротуризм': 0.0,
 '10 балл': 0.0,
 '10 год': 0.0,
 '10 гонка': 0.0,
 '10 градус': 0.0,
 '10 декабрь': 0.0,
 '10 день': 0.0,
 '10 замедление': 0.0,
 '10 килограмм': 3.8513353817064524,
 '10 компания': 3.8513353817064524,
 '10 летие': 0.0,
 '10 летний': 0.9628338454266131,
 '10 месяц': 0.6418892302844087,
 '10 миллиард': 0.0,
 '10 миллион': 3.8513353817064524,
 '10 минута': 3.8513353817064524,
 '10 млн': 0.0,
 '10 млрд': 1.4442507681399195,
 '10 повышать': 3.8513353817064524,
 '10 процедура': 0.0,
 '10 процент': 3.851335381706452

In [52]:
sorted(top.items(), key=lambda item: item[1], reverse=True)

[('кемерово', 123.24273221460648),
 ('престарелый', 123.24273221460648),
 ('дом престарелый', 107.83739068778065),
 ('зеленский сша', 77.02670763412905),
 ('визит зеленский', 65.4727014890097),
 ('престарелый кемерово', 65.4727014890097),
 ('приют', 53.91869534389033),
 ('пожар дом', 46.21602458047742),
 ('гостиница донецк', 42.364689198770975),
 ('месторождение иркутский', 42.364689198770975),
 ('белгород', 38.51335381706453),
 ('кемеровский', 38.51335381706453),
 ('обстрел гостиница', 38.51335381706453),
 ('газопровод чувашия', 34.66201843535807),
 ('поездка зеленский', 34.66201843535807),
 ('зеленский вашингтон', 30.81068305365162),
 ('рогозин', 30.81068305365162),
 ('на', 28.885015362798395),
 ('взрыв газопровод', 26.959347671945164),
 ('газовый месторождение', 26.959347671945164),
 ('египет турист', 26.959347671945164),
 ('закон наказание', 26.959347671945164),
 ('солист группа', 26.959347671945164),
 ('больница донецк', 23.10801229023871),
 ('встречать', 23.10801229023871),
 ('вт

In [53]:
count

4214

In [55]:
X_all = np.asarray(X.sum(axis=0)).ravel()

In [56]:
vocab[np.argsort(X_all)[-100:]]

array(['переговоры', 'показывать', 'день', 'млрд', 'отвечать',
       'запорожский', 'россиянка', 'граница', 'бывший', 'один', 'польша',
       'запад', 'ракета', 'причина', 'президент', 'северный', 'летний',
       'город', 'байден', 'первый', 'ребенок', 'находить', 'днр', 'пожар',
       'госдума', 'мужчина', 'высказываться', 'мид', 'женщина',
       'отказываться', 'рост', 'удар', 'помощь', 'работа', 'дело', 'нато',
       'число', 'поставка', 'обвинять', 'самый', 'взрыв', 'обстрел',
       'германия', 'объяснять', 'свой', 'турция', 'мобилизация', 'кремль',
       'попадать', 'зеленский', 'предупреждать', 'известно', 'видео',
       'дом', 'суд', 'китай', 'начинать', 'получать', 'киев', 'санкция',
       'власть', 'против', 'мир', 'нефть', 'ес', 'способ', 'человек',
       'европа', 'минобороны', 'предлагать', 'регион', 'рубль',
       'призывать', 'цена', 'газ', 'украинский', 'глава', 'раскрывать',
       'оценивать', 'страна', 'военный', 'область', 'сообщать', 'москва',
       'вс

## Hackathon Trend Extractor

In [11]:
get_good_trends(get_all_trends(df, CountVectorizer()))

array(['ребенок', 'учение', 'инспекция', 'днр', 'литва', 'отдых',
       'стоимость', 'июль', 'территория', 'россиянка', 'виза', 'въезд',
       'фбр', 'европа', 'петербург', 'рубль', 'новофедоровка', 'призыв',
       'срок', 'россиянин', 'тайвань', 'трамп', 'обыск', 'аэродром',
       'транзит', 'аэс', 'китай', 'август', 'взрыв', 'крым'], dtype='<U13')

In [10]:
from natasha import Segmenter, NewsMorphTagger, Doc, NewsEmbedding
from sklearn.feature_extraction.text import CountVectorizer

def get_all_trends(corpus, vectorizer):
    X = vectorizer.fit_transform(corpus['processed']).toarray()

    count = 2000

    new_freq = np.asarray(X[:count].sum(axis=0)).ravel() / len(X[:count])
    old_freq = np.asarray(X[count:].sum(axis=0)).ravel() / len(X[count:])

    razn = np.array(new_freq - old_freq)
    vocab = np.array(vectorizer.get_feature_names_out())

    return vocab[np.argsort(razn)[-40:]]


def get_good_trends(result):
    segmenter = Segmenter()
    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)

    trend = []

    for gram in result:

        doc = Doc(gram)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        synt_length = len(doc.tokens)

        if synt_length == 1 and doc.tokens[0].pos == 'NOUN':
            trend.append(gram)

        elif synt_length > 1:
            trend.append(gram)

    return np.array(trend)